In [1]:
!pip install pandas
!pip install openpyxl
!pip install ipywidgets --user
!pip install transformers[torch]

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
import torch

In [3]:
import pandas as pd

In [4]:
data=pd.read_excel('social_media_Data.xlsx')

<ipython-input-4-00cadfd4d2e5>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data=pd.read_excel('social_media_Data.xlsx')


In [4]:
for i in data['Clean_Tweet']:
#     print(type(i))
    if isinstance(i, int)==True:
        print(i)

2022


In [5]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in data['Clean_Tweet']:
    if isinstance(sample, int)==False and isinstance(sample, float)==False:
        sample = sample.replace('\n', '')
        text_data.append(sample)
        if len(text_data) == 10_000:
            # once we git the 10K mark, save to file
            with open(f'dataTextFolder/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
                fp.write('\n'.join(text_data))
            text_data = []
            file_count += 1
# # after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
# with open(f'../../data/text/oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
#     fp.write('\n'.join(text_data))

In [4]:
from pathlib import Path
paths = [str(x) for x in Path('dataTextFolder').glob('**/*.txt')]

In [5]:
paths

['dataTextFolder/text_8.txt',
 'dataTextFolder/text_9.txt',
 'dataTextFolder/text_7.txt',
 'dataTextFolder/text_11.txt',
 'dataTextFolder/text_6.txt',
 'dataTextFolder/text_3.txt',
 'dataTextFolder/text_10.txt',
 'dataTextFolder/text_2.txt',
 'dataTextFolder/text_12.txt',
 'dataTextFolder/text_1.txt',
 'dataTextFolder/text_0.txt',
 'dataTextFolder/text_4.txt',
 'dataTextFolder/text_5.txt']

In [8]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [66]:
# tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
#                 special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

tokenizerBioBERT.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

AttributeError: 'BertTokenizerFast' object has no attribute 'train'

In [91]:
import os

#os.mkdir('./TweetBert')

tokenizerBioBERT.save_model('TweetBert')

AttributeError: 'BertTokenizerFast' object has no attribute 'save_model'

In [36]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('TweetBert', max_len=1024)

In [48]:
#tokens = tokenizer('chris ryan s main conclusion in everest trial: adjuvant everolimus improved rfs in rcc patients after nephrectomy, but nominal significance level not reached obvious example of spin.')

tokens = tokenizerBioBERT('chris ryan s main conclusion in everest trial: adjuvant everolimus improved rfs in rcc patients after nephrectomy, but nominal significance level not reached obvious example of spin.')


In [49]:
print(tokens)

{'input_ids': [101, 22572, 4889, 187, 6582, 188, 1514, 6593, 1107, 1518, 2556, 3443, 131, 8050, 9380, 14194, 1518, 11014, 6308, 4725, 187, 22816, 1107, 187, 19515, 4420, 1170, 24928, 7880, 1874, 5822, 18574, 117, 1133, 16251, 7467, 1634, 1136, 1680, 5119, 1859, 1104, 6898, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [50]:
tokens.input_ids

[101,
 22572,
 4889,
 187,
 6582,
 188,
 1514,
 6593,
 1107,
 1518,
 2556,
 3443,
 131,
 8050,
 9380,
 14194,
 1518,
 11014,
 6308,
 4725,
 187,
 22816,
 1107,
 187,
 19515,
 4420,
 1170,
 24928,
 7880,
 1874,
 5822,
 18574,
 117,
 1133,
 16251,
 7467,
 1634,
 1136,
 1680,
 5119,
 1859,
 1104,
 6898,
 119,
 102]

In [13]:
for i in paths[:7]:
    print(i)

dataTextFolder/text_8.txt
dataTextFolder/text_9.txt
dataTextFolder/text_7.txt
dataTextFolder/text_11.txt
dataTextFolder/text_6.txt
dataTextFolder/text_3.txt
dataTextFolder/text_10.txt


In [15]:
len(paths)

13

In [13]:
for i in paths[:7]:
    with open(i, 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
        batch = tokenizerBioBERT(lines, max_length=512, padding='max_length', truncation=True)
        labels = torch.tensor([x for x in batch.input_ids])
        mask = torch.tensor([x for x in batch.attention_mask])
        # make copy of labels tensor, this will be input_ids
        input_ids = labels.detach().clone()
        # create random array of floats with equal dims to input_ids
        rand = torch.rand(input_ids.shape)
        # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
        mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
        # loop through each row in input_ids tensor (cannot do in parallel)
        for i in range(input_ids.shape[0]):
            # get indices of mask positions from mask array
            selection = torch.flatten(mask_arr[i].nonzero()).tolist()
            # mask input_ids
            input_ids[i, selection] = 3  # our custom [MASK] token == 3
        encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}
        dataset = Dataset(encodings)
        loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
        epochs = 2

        for epoch in range(epochs):
            # setup loop with TQDM and dataloader
            #loop = tqdm(loader, leave=True)
            for batch in loader:
                # initialize calculated gradients (from prev step)
                optim.zero_grad()
                # pull all tensor batches required for training
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                # process
                outputs = modelBioBERT(input_ids, attention_mask=attention_mask,
                                labels=labels)
                # extract loss
                loss = outputs.loss
                # calculate loss for every parameter that needs grad update
                loss.backward()
                # update parameters
                optim.step()
                # print relevant info to progress bar
        #         loop.set_description(f'Epoch {epoch}')
        #   loop.set_postfix(loss=loss.item())


In [52]:
len(lines)

10000

In [71]:
#batch = tokenizer(lines, max_length=1024, padding='max_length', truncation=True)
batch = tokenizerBioBERT(lines, max_length=512, padding='max_length', truncation=True)


len(batch)

3

In [54]:
import pandas as pd
tmp = pd.DataFrame.from_dict(batch)

In [55]:
tmp

,0
0,input_ids
1,token_type_ids
2,attention_mask


In [72]:
print(len(batch['input_ids'][0]))

512


In [73]:
print(len(batch['input_ids']))

10000


In [74]:
import torch

labels = torch.tensor([x for x in batch.input_ids])
mask = torch.tensor([x for x in batch.attention_mask])

In [75]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [76]:
input_ids.shape


torch.Size([10000, 512])

In [77]:
input_ids[10][:200]


tensor([  101,  5415,     3,  2593,  1108,  2628,  1114,     3,     3,  1107,
          185,  2145,     3,   182, 19878,  1665,     3,  1114, 11437,  6005,
          116, 10347,  1186,     3,  1112,  2528, 20581,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [78]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}


In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [80]:
dataset = Dataset(encodings)


In [81]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizerBioBERT = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

modelBioBERT = AutoModelForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=1024,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [28]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
modelBioBERT.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [11]:
from transformers import AdamW

# activate training mode
#model.train()
modelBioBERT.train()
# initialize optimizer
optim = AdamW(modelBioBERT.parameters(), lr=1e-4)

In [84]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    #loop = tqdm(loader, leave=True)
    for batch in loader:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = modelBioBERT(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
#         loop.set_description(f'Epoch {epoch}')
#   loop.set_postfix(loss=loss.item())

In [14]:
modelBioBERT.save_pretrained('./TweetBert') 


In [87]:
loss

tensor(0.0233, grad_fn=<NllLossBackward0>)

In [38]:
# outputs

In [88]:
from transformers import pipeline

In [92]:
#fill = pipeline('fill-mask', model='TweetBert', tokenizer='TweetBert')
#tokenizerBioBERT
fill = pipeline('fill-mask', model='TweetBert', tokenizer=tokenizerBioBERT)


In [93]:
#fill(f'ciao {fill.tokenizer.mask_token} va?')
fill(f'''quote of the week at #aacr22: "it's so good to be here in {fill.tokenizer.mask_token}, not just in silico"quote of the week at #aacr22: "it's so good to be here in vivo, not just in silico"''')
# quote of the week at #aacr22: "it's so good to be here in vivo, not just in silico"quote of the week at #aacr22: "it's so good to be here in vivo, not just in silico"

[{'score': 0.23521080613136292,
  'token': 1825,
  'token_str': 'person',
  'sequence': 'quote of the week at # aacr22 : " it\'s so good to be here in person, not just in silico " quote of the week at # aacr22 : " it\'s so good to be here in vivo, not just in silico "'},
 {'score': 0.0625469908118248,
  'token': 2415,
  'token_str': 'practice',
  'sequence': 'quote of the week at # aacr22 : " it\'s so good to be here in practice, not just in silico " quote of the week at # aacr22 : " it\'s so good to be here in vivo, not just in silico "'},
 {'score': 0.04964900016784668,
  'token': 191,
  'token_str': 'v',
  'sequence': 'quote of the week at # aacr22 : " it\'s so good to be here in v, not just in silico " quote of the week at # aacr22 : " it\'s so good to be here in vivo, not just in silico "'},
 {'score': 0.03765760362148285,
  'token': 2000,
  'token_str': 'space',
  'sequence': 'quote of the week at # aacr22 : " it\'s so good to be here in space, not just in silico " quote of the w

In [95]:
fill(f''' these apps are a game changer for #ibs as everyone now has {fill.tokenizer.mask_token} to fda-approved treatments #acg2022 ''')

[{'score': 0.991570234298706,
  'token': 2469,
  'token_str': 'access',
  'sequence': 'these apps are a game changer for # ibs as everyone now has access to fda - approved treatments # acg2022'},
 {'score': 0.0014464546693488955,
  'token': 7401,
  'token_str': 'exposure',
  'sequence': 'these apps are a game changer for # ibs as everyone now has exposure to fda - approved treatments # acg2022'},
 {'score': 0.000869302952196449,
  'token': 5133,
  'token_str': 'responded',
  'sequence': 'these apps are a game changer for # ibs as everyone now has responded to fda - approved treatments # acg2022'},
 {'score': 0.0005974660161882639,
  'token': 6759,
  'token_str': 'switched',
  'sequence': 'these apps are a game changer for # ibs as everyone now has switched to fda - approved treatments # acg2022'},
 {'score': 0.0005652213003486395,
  'token': 4762,
  'token_str': 'committed',
  'sequence': 'these apps are a game changer for # ibs as everyone now has committed to fda - approved treatment